In [1]:
import pandas as pd
from dateutil.parser import parse

In [2]:
mwdf = pd.read_csv('hrl_load_metered - 20170201-20200131.csv')
temp_df = pd.read_csv('hr_temp_20170201-20200131_subset.csv')
temp_df2 = pd.read_csv('hr_temp_20200201-20200229_subset.csv')

In [3]:
temp_df.isna().sum() 

STATION                      0
DATE                         0
REPORT_TYPE                  0
SOURCE                       0
HourlyDryBulbTemperature    37
dtype: int64

In [4]:
temp_df['DATE'] = temp_df['DATE'].apply(parse).dt.round('H')
temp_df.sort_values(by="DATE", inplace=True)
temp_df['HourlyDryBulbTemperature'].interpolate(inplace=True)
# temp_df = temp_df.astype({'HourlyDryBulbTemperature': 'int32'})
temp_df.head()

,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
0,72520514762,2017-02-01 01:00:00,FM-15,7,37.0
1,72520514762,2017-02-01 02:00:00,FM-15,7,37.0
2,72520514762,2017-02-01 03:00:00,FM-15,7,36.0
3,72520514762,2017-02-01 04:00:00,FM-15,7,36.0
4,72520514762,2017-02-01 05:00:00,FM-15,7,36.0


In [5]:
# We've been told these values are not important
temp_df.drop(['STATION', 'REPORT_TYPE', 'SOURCE'], axis=1, inplace=True)

In [6]:
temp_df2['DATE'] = temp_df2['DATE'].apply(parse).dt.round('H')
temp_df2.sort_values(by="DATE", inplace=True)
temp_df2['HourlyDryBulbTemperature'].interpolate(inplace=True)
temp_df2 = temp_df2.astype({'HourlyDryBulbTemperature': 'int32'})
# We've been told these values are not important
temp_df2.drop(['STATION', 'REPORT_TYPE', 'SOURCE'], axis=1, inplace=True)
temp_df2.head()

,DATE,HourlyDryBulbTemperature
0,2020-02-01 01:00:00,32
1,2020-02-01 02:00:00,32
2,2020-02-01 03:00:00,32
3,2020-02-01 04:00:00,32
4,2020-02-01 05:00:00,31


In [7]:
mwdf['datetime_beginning_utc'] = mwdf['datetime_beginning_utc'].apply(parse)
mwdf['datetime_beginning_ept'] = mwdf['datetime_beginning_ept'].apply(parse)

mwdf.head()

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,2017-02-01 05:00:00,2017-02-01 00:00:00,RFC,WEST,DUQ,DUQ,1419.881,True
1,2017-02-01 06:00:00,2017-02-01 01:00:00,RFC,WEST,DUQ,DUQ,1379.505,True
2,2017-02-01 07:00:00,2017-02-01 02:00:00,RFC,WEST,DUQ,DUQ,1366.106,True
3,2017-02-01 08:00:00,2017-02-01 03:00:00,RFC,WEST,DUQ,DUQ,1364.453,True
4,2017-02-01 09:00:00,2017-02-01 04:00:00,RFC,WEST,DUQ,DUQ,1391.265,True


In [8]:
# Are the date fields the same for joining on?
len(set(mwdf['datetime_beginning_utc'].tolist()) & set(temp_df['DATE'].tolist() ))== len(set(mwdf['datetime_beginning_ept'].tolist()) & set(temp_df['DATE'].tolist() ))

True

In [9]:
mwdf['year'] = pd.DatetimeIndex(mwdf['datetime_beginning_utc']).year
mwdf['month'] = pd.DatetimeIndex(mwdf['datetime_beginning_utc']).month
mwdf['weekday'] = pd.DatetimeIndex(mwdf['datetime_beginning_utc']).weekday
mwdf['hour'] = pd.DatetimeIndex(mwdf['datetime_beginning_utc']).hour
mwdf.drop(['datetime_beginning_ept', 'nerc_region', 'mkt_region', 'zone', 'load_area', 'is_verified'], axis=1, inplace=True)

### Some extreme values look suspicious, maybe we should drop them

In [10]:
temp_df['HourlyDryBulbTemperature'].value_counts()

68.000000    713
70.000000    671
67.000000    655
69.000000    638
66.000000    592
            ... 
29.896552      1
92.000000      1
64.500000      1
71.500000      1
28.551724      1
Name: HourlyDryBulbTemperature, Length: 135, dtype: int64

In [11]:
len(mwdf), len(temp_df), len(temp_df2)

(26280, 26280, 696)

In [12]:
mwdf['DATE'] = mwdf['datetime_beginning_utc']
merged = mwdf.set_index('DATE').join(temp_df.set_index('DATE'),   rsuffix='_o')

# merged = mwdf.join(temp_df.set_index('DATE'),   rsuffix='_o')
# #mwdf['datetime_beginning_utc'] 
merged

,datetime_beginning_utc,mw,year,month,weekday,hour,HourlyDryBulbTemperature
DATE,,,,,,,
2017-02-01 05:00:00,2017-02-01 05:00:00,1419.881,2017,2,2,5,36.0
2017-02-01 06:00:00,2017-02-01 06:00:00,1379.505,2017,2,2,6,36.0
2017-02-01 07:00:00,2017-02-01 07:00:00,1366.106,2017,2,2,7,34.0
2017-02-01 08:00:00,2017-02-01 08:00:00,1364.453,2017,2,2,8,33.0
2017-02-01 09:00:00,2017-02-01 09:00:00,1391.265,2017,2,2,9,34.0
...,...,...,...,...,...,...,...
2020-02-01 00:00:00,2020-02-01 00:00:00,1618.484,2020,2,5,0,34.0
2020-02-01 01:00:00,2020-02-01 01:00:00,1580.925,2020,2,5,1,NaN
2020-02-01 02:00:00,2020-02-01 02:00:00,1545.354,2020,2,5,2,NaN


In [13]:
merged.isna().sum()

datetime_beginning_utc      0
mw                          0
year                        0
month                       0
weekday                     0
hour                        0
HourlyDryBulbTemperature    4
dtype: int64

In [14]:
temp_df.isna().sum()

DATE                        0
HourlyDryBulbTemperature    0
dtype: int64

In [15]:
# There's probably an easier way
updates =[]
for idx, row in merged.iterrows():
    if pd.isna(row['HourlyDryBulbTemperature']):
        the_date = row['datetime_beginning_utc']
        the_val = temp_df2.query(f' DATE=="{the_date}"')['HourlyDryBulbTemperature'].item()
        updates.append( (idx, the_val))
for idx, val in updates:
    merged.loc[idx, 'HourlyDryBulbTemperature'] =val
merged

,datetime_beginning_utc,mw,year,month,weekday,hour,HourlyDryBulbTemperature
DATE,,,,,,,
2017-02-01 05:00:00,2017-02-01 05:00:00,1419.881,2017,2,2,5,36.0
2017-02-01 06:00:00,2017-02-01 06:00:00,1379.505,2017,2,2,6,36.0
2017-02-01 07:00:00,2017-02-01 07:00:00,1366.106,2017,2,2,7,34.0
2017-02-01 08:00:00,2017-02-01 08:00:00,1364.453,2017,2,2,8,33.0
2017-02-01 09:00:00,2017-02-01 09:00:00,1391.265,2017,2,2,9,34.0
...,...,...,...,...,...,...,...
2020-02-01 00:00:00,2020-02-01 00:00:00,1618.484,2020,2,5,0,34.0
2020-02-01 01:00:00,2020-02-01 01:00:00,1580.925,2020,2,5,1,32.0
2020-02-01 02:00:00,2020-02-01 02:00:00,1545.354,2020,2,5,2,32.0


In [16]:
merged.dtypes

datetime_beginning_utc      datetime64[ns]
mw                                 float64
year                                 int64
month                                int64
weekday                              int64
hour                                 int64
HourlyDryBulbTemperature           float64
dtype: object

In [18]:
merged.rename(columns={"datetime_beginning_utc": "date",
                       "HourlyDryBulbTemperature": "temp"}, errors="raise")

,date,mw,year,month,weekday,hour,temp
DATE,,,,,,,
2017-02-01 05:00:00,2017-02-01 05:00:00,1419.881,2017,2,2,5,36.0
2017-02-01 06:00:00,2017-02-01 06:00:00,1379.505,2017,2,2,6,36.0
2017-02-01 07:00:00,2017-02-01 07:00:00,1366.106,2017,2,2,7,34.0
2017-02-01 08:00:00,2017-02-01 08:00:00,1364.453,2017,2,2,8,33.0
2017-02-01 09:00:00,2017-02-01 09:00:00,1391.265,2017,2,2,9,34.0
...,...,...,...,...,...,...,...
2020-02-01 00:00:00,2020-02-01 00:00:00,1618.484,2020,2,5,0,34.0
2020-02-01 01:00:00,2020-02-01 01:00:00,1580.925,2020,2,5,1,32.0
2020-02-01 02:00:00,2020-02-01 02:00:00,1545.354,2020,2,5,2,32.0


In [19]:
merged = merged.astype({'HourlyDryBulbTemperature': 'int64'})

In [22]:
print("%s training rows" % len(merged.query(f"DATE < '{parse('2020-01-01')}'"))   )
merged.query(f"DATE < '{parse('2020-01-01')}'").to_csv('timeseries_train.csv')

25531 training rows


In [23]:
print("%s test rows" % len(merged.query(f"DATE > '{parse('2020-01-01')}'")))
merged.query(f"DATE > '{parse('2020-01-01')}'").to_csv('timeseries_test.csv')

748 test rows
